In [1]:
import pandas as pd, math, numpy as np
import sys 
sys.path.append('C:\\Users\\Nuo.Chen\\PycharmProjects\\Imbalance_price_forecast')
from utils.import_packages import *
from data_gathering import *
from features.features import *

# clean power forecast data

## clean multiple forecast for the same period

In [30]:
solar_df = pd.read_excel(weather_folder + '/solar-forecast.xlsx')
solar_df.tail()

,TIMESTAMP_UTC,VENDOR_ID,PERIOD,value,maxvalue,minvalue,CREATED_TIMESTAMP_UTC,COUNTRY_ID
14417,2018-06-20 21:00:00,MKON,H,42.383018,46.5,27.0,2018-06-19 08:25:00,NL
14418,2018-06-20 22:00:00,MKON,H,1.384615,1.8,0.6,2018-06-19 00:27:00,NL
14419,2018-06-20 22:00:00,MKON,H,1.457547,1.8,0.6,2018-06-19 08:25:00,NL
14420,2018-06-20 23:00:00,MKON,H,0.000000,0.0,0.0,2018-06-19 00:27:00,NL
14421,2018-06-20 23:00:00,MKON,H,0.000000,0.0,0.0,2018-06-19 08:25:00,NL


In [29]:
solar_df = pd.read_excel(weather_folder + '/solar-forecast.xlsx')
clean_solar_df = clean_multiple_forecast(solar_df, 'TIMESTAMP_UTC', 'CREATED_TIMESTAMP_UTC')
clean_solar_df.to_excel(weather_folder + '/clean-solar-forecast.xlsx')

wind_df = pd.read_excel(weather_folder + '/wind-forecast.xlsx')
clean_wind_df = clean_multiple_forecast(wind_df, 'TIMESTAMP_UTC', 'CREATED_TIMESTAMP_UTC')
clean_wind_df.to_excel(weather_folder + '/clean-wind-forecast.xlsx')

## change to local timezone


In [31]:
solar_df = pd.read_excel(weather_folder + '/clean-solar-forecast.xlsx')
wind_df = pd.read_excel(weather_folder + '/clean-wind-forecast.xlsx')

In [64]:
import pytz

def convert_2_utc( localtime, timezone = 'Europe/Amsterdam'):
    utc = pytz.utc
    amsterdam = pytz.timezone(timezone)
    
    am_dt = [amsterdam.localize(t) for t in localtime]
    utc_time = [t.astimezone(utc) for t in am_dt]
    
    return utc_time

def convert_2_local( utc_time, timezone = 'Europe/Amsterdam'):
    local_time = [pytz.utc.localize(t) for t in utc_time]
    am_dt = [t.astimezone(timezone) for t in local_time]
    
    return am_dt

In [73]:
solar_df['Datetime'] = convert_2_local( solar_df['TIMESTAMP_UTC'])
wind_df['Datetime'] = convert_2_local( wind_df['TIMESTAMP_UTC'])

## complete the date&time list per PTE for each forecasting data

In [80]:
start_date = datetime(2015,9,23)
end_date = datetime(2018,6,20)
empty_time_df = make_date_time_full_list( start_date,end_date,interval={'minutes':15})
select_forecast_feat = ['DateTime', 'Date', 'PTE', 'value', 'maxvalue', 'minvalue']

complete_solar_df = empty_time_df.merge(clean_solar_df, left_on = 'DateTime', right_on = 'TIMESTAMP_UTC', how = 'left')
complete_solar_df = complete_solar_df[select_forecast_feat]

complete_wind_df = empty_time_df.merge(clean_wind_df, left_on = 'DateTime', right_on = 'TIMESTAMP_UTC', how = 'left')
complete_wind_df = complete_wind_df[select_forecast_feat]

common_col = ['DateTime', 'Date', 'PTE']
complete_solar_df.columns = common_col + ['solar_value', 'solar_maxvalue','solar_minvalue']
complete_wind_df.columns = common_col + ['wind_value', 'wind_maxvalue', 'wind_minvalue']

complete_solar_df.to_excel(weather_folder + '/compete_solar_df.xlsx', index = False)
complete_wind_df.to_excel(weather_folder + '/compete_wind_df.xlsx', index = False)


TypeError: 'module' object is not callable

## merge soloar and wind 

In [ ]:
solar_wind_df = complete_solar_df.merge(complete_wind_df, on=['DateTime', 'Date', 'PTE'], how='inner')
solar_wind_df.to_excel(weather_folder + '/complete_solar_wind.xlsx', index = False)

In [30]:
complete_solar_df = pd.read_excel(weather_folder + '/compete_solar_df.xlsx')
complete_wind_df = pd.read_excel(weather_folder + '/compete_wind_df.xlsx')

solar_wind_df = pd.read_excel(weather_folder + '/complete_solar_wind.xlsx')

In [34]:
complete_wind_df.head()

,DateTime,Date,PTE,wind_value,wind_maxvalue,wind_minvalue
0,2015-09-23 00:00:00,2015-09-23,1,108.43,129.7,90.6
1,2015-09-23 00:15:00,2015-09-23,2,NaN,NaN,NaN
2,2015-09-23 00:30:00,2015-09-23,3,NaN,NaN,NaN
3,2015-09-23 00:45:00,2015-09-23,4,NaN,NaN,NaN
4,2015-09-23 01:00:00,2015-09-23,5,119.21,152.9,101.4


In [29]:
def check_duplicate_rows(df, onColumns):
    duplicate = df.duplicated(subset = onColumns)
    return duplicate[duplicate == True]

check_duplicate_rows(complete_wind_df, ['DateTime'])
check_duplicate_rows()

Series([], dtype: bool)

In [ ]:
complete_solar_df = pd.read_excel(weather_folder + '/compete_solar_df.xlsx')
complete_wind_df = pd.read_excel(weather_folder + '/compete_wind_df.xlsx')

solar_wind_df = complete_solar_df.merge(complete_wind_df, on=['DateTime', 'Date', 'PTE'], how='inner')
solar_wind_df.to_excel(weather_folder + '/complete_solar_wind.xlsx', index = False)

## impute

impute strategy:
1. the forecast is per hour, then evenly shared the power output for the four quarters, including value and max-min value
2. dicard data from 2016-12-12 to 2017-12-26
3. impute the other missing dates with previous date data

In [22]:
solar_wind_df = pd.read_excel(weather_folder + '/clean_solar_wind_df.xlsx', index = False)

In [23]:
# for testing
solar_wind_df.loc[3080:3085]

,DateTime,Date,PTE,solar_value,solar_maxvalue,solar_minvalue,wind_value,wind_maxvalue,wind_minvalue
3080,2015-10-25 02:00:00,2015-10-25,9,0.0,0.0,0.0,0.0,0.0,0.0
3081,2015-10-25 02:00:00,2015-10-25,9,0.0,0.0,0.0,0.0,0.0,0.0
3082,2015-10-25 02:00:00,2015-10-25,9,0.0,0.0,0.0,0.0,0.0,0.0
3083,2015-10-25 02:00:00,2015-10-25,9,0.0,0.0,0.0,0.0,0.0,0.0
3084,2015-10-25 02:15:00,2015-10-25,10,NaN,NaN,NaN,NaN,NaN,NaN
3085,2015-10-25 02:30:00,2015-10-25,11,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
solar_wind_df = pd.read_excel(weather_folder + '/clean_solar_wind_df.xlsx', index = False)
cur_index = 0

print('imputing...')

while cur_index < len(solar_wind_df):

    PTE = solar_wind_df.loc[cur_index]['PTE']
    cur_row = solar_wind_df.loc[cur_index][solar_wind_col]

    if PTE % 4 == 1 & cur_row.isnull().any() & cur_index >=96:
        #         cur_date = solar_wind_df.loc[cur_index]['Date']
        #         if cur_date.isoweekday() <= 5:
        #             invalid_missing_dates.append(cur_date)

        # impute missing day
        last_day_data = solar_wind_df.loc[cur_index - 96][solar_wind_col]
        impute_data = last_day_data
        solar_wind_df.loc[cur_index, solar_wind_col] = impute_data

    elif PTE % 4 == 1:
        # impute missing PTE
        impute_data = cur_row

    else:
        raise ValueError('Invalid missing values')

    solar_wind_df.loc[cur_index + 1, solar_wind_col] = impute_data
    solar_wind_df.loc[cur_index + 2, solar_wind_col] = impute_data
    solar_wind_df.loc[cur_index + 3, solar_wind_col] = impute_data
    cur_index += 4

print('finish imputation')

In [82]:
solar_wind_col = ['solar_value', 'solar_maxvalue', 'solar_minvalue', 'wind_value', 'wind_maxvalue', 'wind_minvalue']
solar_wind_df = pd.read_excel(weather_folder + '/complete_solar_wind.xlsx')

solar_wind_df = solar_wind_df[(solar_wind_df['Date']< datetime(2016,12,12))|(solar_wind_df['Date']> datetime(2017,12,26))]
solar_wind_df[ solar_wind_col ]/=4

# deleted missing date in 2017 and averaged the hourly data
solar_wind_df.to_excel(weather_folder + '/clean_solar_wind_df.xlsx', index = False)

TypeError: 'module' object is not callable

In [ ]:
solar_wind_df = pd.read_excel(weather_folder + '/clean_solar_wind_df.xlsx')
solar_wind_col = ['solar_value', 'solar_maxvalue', 'solar_minvalue', 'wind_value', 'wind_maxvalue', 'wind_minvalue']

cur_index = 0

start = time.time()
# invalid_missing_dates = list()


while cur_index < len(solar_wind_df):

    PTE = solar_wind_df.loc[cur_index]['PTE']
    cur_row = solar_wind_df.loc[cur_index][solar_wind_col]
    
    if PTE % 4 == 1 & cur_row.isnull().any():
#         cur_date = solar_wind_df.loc[cur_index]['Date']
#         if cur_date.isoweekday() <= 5:
#             invalid_missing_dates.append(cur_date)

        # impute missing day
        last_day_data = solar_wind_df.loc[cur_index-96][solar_wind_col]
        impute_data = last_day_data
        solar_wind_df.loc[cur_index, solar_wind_col] = impute_data
        
    elif PTE % 4 ==1:
        # impute missing PTE
        impute_data = cur_row

    else:
        raise ValueError('Invalid missing values')
    
    solar_wind_df.loc[cur_index+1, solar_wind_col] = impute_data
    solar_wind_df.loc[cur_index+2, solar_wind_col] = impute_data
    solar_wind_df.loc[cur_index+3, solar_wind_col] = impute_data
    cur_index += 4

    
# print('invalid missing dates(not weekend)')
# print(set(invalid_missing_dates))
end = time.time()

print('takes:{} min'.format((end - start)/60))

In [ ]:
solar_wind_df.to_excel(weather_folder + '/imputed_solar_wind.xlsx', index = False)

# merge day-ahead price data

Note: this column couldn't be used as a feature for imbalance price prediction

Data is merged by the delivery date and PTE

In [2]:
solar_wind_df = pd.read_excel(weather_folder+'/imputed_solar_wind.xlsx')

In [12]:
day_ahead_price_df = pd.read_excel(day_ahead_folder + '/new-DA-price.xlsx')

In [18]:
# ## get average day-ahead-price for each PTE
# DA_price = day_ahead_price_df.groupby(['DeliveryDate','start'], as_index=False)['Value'].mean()
# merged_DA = day_ahead_price_df.merge(DA_price, on = ['DeliveryDate', 'start'],\
#                                                 how = 'inner')
# merged_DA = merged_DA.drop(['Value_x'], axis =1)
# merged_DA = merged_DA.drop_duplicates()
# merged_DA.columns = ['Date', 'PTE', 'DA-price']
# merged_DA['Date'] = merged_DA['Date'].astype('datetime64[ns]')

# merge power data
day_ahead_price_df.columns = ['Date','DA-price', 'PTE']
DA_power_df = day_ahead_price_df.merge(solar_wind_df, on = ['Date', 'PTE'], how = 'inner')
DA_power_df.head()

,Date,DA-price,PTE,DateTime,solar_value,solar_maxvalue,solar_minvalue,wind_value,wind_maxvalue,wind_minvalue
0,2016-01-01,23.86,1,2016-01-01 00:00:00,0.0,0.0,0.0,297.2600,318.175,267.225
1,2016-01-01,23.86,2,2016-01-01 00:15:00,0.0,0.0,0.0,297.2600,318.175,267.225
2,2016-01-01,23.86,3,2016-01-01 00:30:00,0.0,0.0,0.0,297.2600,318.175,267.225
3,2016-01-01,23.86,4,2016-01-01 00:45:00,0.0,0.0,0.0,297.2600,318.175,267.225
4,2016-01-01,22.39,5,2016-01-01 01:00:00,0.0,0.0,0.0,281.0725,299.825,259.450


In [21]:
DA_power_df.to_excel(data_folder_path + '/temp/day_ahead_merge_power.xlsx', index = False)

# merge imbalance price

In [22]:
merged_DA_power_df = pd.read_excel(data_folder_path + '/temp/day_ahead_merge_power.xlsx')

In [23]:
imbalance_df = pd.read_csv(data_folder_path + '/imbalance/tennet16to18.csv')

In [24]:
imb_df = imbalance_df[['week', 'Date', 'PTE', \
                       'take_from_system_kWhPTE','feed_into_system_EURMwh',\
                       'purchase_kWhPTE','sell_kWhPTE', 'absolute_kWhPTE']]
imb_df.columns = ['week', 'Date', 'PTE', \
                  'take_from_system_price', 'feed_into_system_price', \
                  'system_purchase_vol', 'system_sell_vol', 'system_absolute_vol']
imb_df['Date'] = imb_df['Date'].astype('datetime64[ns]')

In [25]:
merge_DA_imb_power_df = imb_df.merge(merged_DA_power_df, on = ['Date', 'PTE'],\
                                     how = 'inner')

In [26]:
merge_DA_imb_power_df.to_excel(data_folder_path + '/temp/merged_DA_imb_power_df.xlsx', index = False)

# trade data is unavailable before 2017. no use for now. 

In [14]:
merge_DA_imb_power_df.to_excel(data_folder_path + '/final-merged-df.xlsx', index = False)

NameError: name 'merge_DA_imb_power_df' is not defined

# merge demand forecast

In [44]:
position_df = pd.read_excel('../data_gathering/data' +'/position/nordzee-wind-pnl.xlsx', encoding = 'ISO-8859-1')

,DeliveryDate,PERIOD,First_Forecast_Volume,ActualVolumes,Feed_Into,Take_From,DayAheadPrice,MAE,Diff,DayAheadPnL,ImbalanceCost,TotalPnL,Month
0,2018-01-01,1,21694.08426,21777,-57.54,58.62,27.20,0.003807,-82.91574,2.255308,-4.860521,-2.605213,1
1,2018-01-01,2,21694.71485,21155,37.02,37.02,27.20,0.025512,539.71485,-14.680244,19.980244,5.300000,1
2,2018-01-01,3,21695.28868,21046,33.10,33.10,27.20,0.030851,649.28868,-17.660652,21.491455,3.830803,1
3,2018-01-01,4,21695.81087,21754,33.10,33.10,27.20,0.002675,-58.18913,1.582744,-1.926060,-0.343316,1
4,2018-01-01,5,21696.28606,21684,33.09,33.09,27.30,0.000567,12.28606,-0.335409,0.406546,0.071136,1
5,2018-01-01,6,21697.28855,21456,35.19,35.19,27.30,0.011246,241.28855,-6.587177,8.490944,1.903767,1
6,2018-01-01,7,21698.13111,21226,33.56,33.56,27.30,0.022243,472.13111,-12.889179,15.844720,2.955541,1
7,2018-01-01,8,21698.69306,21524,26.09,26.09,27.30,0.008116,174.69306,-4.769121,4.557742,-0.211379,1
8,2018-01-01,9,21698.84228,21747,36.62,36.62,30.10,0.002214,-48.15772,1.449547,-1.763536,-0.313988,1
9,2018-01-01,10,21697.85733,21674,32.45,32.45,30.10,0.001101,23.85733,-0.718106,0.774170,0.056065,1


In [15]:
merged_df = pd.read_excel(data_folder_path + '/final-merged-df.xlsx')
demand_df = pd.read_excel(data_folder_path + '/demand/demand-forecast.xlsx')

In [ ]:
final_df = merged_df.merge()